In [1]:
import pyttsx3
import datetime
import speech_recognition as sr
import pyautogui
import pyjokes
import pywhatkit
import wikipedia
import os
import cv2
from pyowm import OWM
from pyowm.utils.config import get_default_config
from pyowm.utils import timestamps
from PIL import Image, ImageDraw, ImageFont
from fpdf import FPDF
import random

# Initialize pyttsx3 for speech synthesis
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)
r = sr.Recognizer()

def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def time():
    speak("The time is: ")
    t = datetime.datetime.now().strftime("%I:%M:%S")
    speak(t)

def date():
    now = datetime.datetime.now()
    current_date = now.strftime("%B %d, %Y")
    speak(f"Today's date is {current_date}")

def play_music(song):
    pywhatkit.playonyt(song)
    speak(f"Playing {song} on YouTube")

def joke():
    joke_text = pyjokes.get_joke()
    print(joke_text)
    speak(joke_text)

def screenshot():
    img = pyautogui.screenshot()
    img.save(r"D:/jarvis/ss.png")
    speak("Screenshot taken")

def weather(location):
    # Setup for OpenWeatherMap API
    config = get_default_config()
    config['language'] = 'en'
    owm = OWM('your-api-key')  # Insert your API key here

    mgr = owm.weather_manager()
    observation = mgr.weather_at_place(location)
    weather = observation.weather

    temperature = weather.temperature('celsius')["temp"]
    speak(f"The current temperature in {location} is {temperature} degrees Celsius.")

def command():
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language="en-in")
        print(query)
    except Exception as e:
        print(e)
        speak("Sorry, I couldn't hear you. Could you say that again?")
        return "None"
    return query

def take_picture():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Navin webcam")
    c = 0
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed")
            break
        cv2.imshow("Test", frame)
        k = cv2.waitKey(1)
        q = command()
        if "close" in q:
            print("Closing the camera")
            speak("Camera closed")
            break
        elif "take" in q or "picture" in q:
            img = f"pic{c}.png"
            cv2.imwrite(img, frame)
            print("Picture taken")
            c += 1
            speak("Picture captured!")
        elif "offline" in q:
            quit()

if _name_ == "_main_":
    speak('Hello, I am your Virtual Buddy.')
    speak('How can I help you today?')

    while True:
        query = command()
        if query is not None:
            query = query.lower()

        if 'time' in query:
            time()

        elif 'date' in query:
            date()

        elif 'wikipedia' in query or 'wiki' in query:
            speak("What should I search on Wikipedia?")
            a = command()
            speak("Searching...")
            result = wikipedia.summary(a, sentences=2)
            print(result)
            speak(result)

        elif 'chrome' in query:
            speak('What should I search in chrome?')
            search = command().lower()
            wb.open(search+'.com')

        elif 'weather' in query:
            speak("Please tell me the location for the weather update.")
            location = command().lower()
            weather(location)

        elif "youtube" in query:
            speak("Which video do you want to play?")
            song = command()
            play_music(song)

        elif 'joke' in query:
            joke()

        elif 'screenshot' in query:
            screenshot()

        elif 'camera' in query:
            take_picture()

        elif 'offline' in query:
            speak("Goodbye!")
            break